In [1]:
# Import Dependicies 
import numpy as np
import pandas as pd
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
# Create Engine,use automap_base function, and create new database using reflect feature 
engine = create_engine("sqlite:///hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect = True)


In [3]:
# Obtain classes keys
Base.classes.keys()


['measurement', 'station']

In [4]:
# Saving reference for each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
# List different keys and columns from Measurement table 
Measurement.__table__.columns.keys()

['id', 'station', 'date', 'prcp', 'tobs']

In [6]:
# List different keys and columns from Station table 
Station.__table__.columns.keys()

['id', 'station', 'name', 'latitude', 'longitude', 'elevation']

In [7]:
# Create session 
session = Session(engine)

In [8]:
# Create year list spanning from start to end of database 
min_date = session.query(func.min(Measurement.date)).scalar()
max_date = session.query(func.max(Measurement.date)).scalar()

print(f" Min date is {min_date}, and Max date is {max_date}.")

year_range = np.arange(2010, 2018,1)
year_range

 Min date is 2010-01-01, and Max date is 2017-08-23.


array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [9]:
# Create database for weather data for each year 
results_1 = session.query(Measurement.station, Measurement.date, Measurement.prcp, Measurement.tobs).all()
results_1

weather_df = pd.DataFrame(results_1, columns = ["station_id", "date", "precipitation (inch)", "temperature (F)"])
weather_df["date"] = pd.to_datetime(weather_df["date"])
# weather_df.set_index("date", inplace = True)
# weather_df = weather_df.sort_index()
weather_df.head()

,station_id,date,precipitation (inch),temperature (F)
0,USC00519397,2010-01-01,0.08,65.0
1,USC00519397,2010-01-02,0.00,63.0
2,USC00519397,2010-01-03,0.00,74.0
3,USC00519397,2010-01-04,0.00,76.0
4,USC00519397,2010-01-06,NaN,73.0


In [10]:
# Create database for June weather 
month = dt.datetime(2010,6,1).month

june_df = pd.DataFrame(columns = ["station_id", "date", "precipitation (inch)", "temperature (F)"])

for index in np.arange(0, len(weather_df), 1):
    if weather_df.iloc[index]['date'].month == month:
        row = weather_df.iloc[index]
        june_df = june_df.append(row)
    else: 
        june_df = june_df
        
june_df.head()

,station_id,date,precipitation (inch),temperature (F)
133,USC00519397,2010-06-01,0.00,78.0
134,USC00519397,2010-06-02,0.01,76.0
135,USC00519397,2010-06-03,0.00,78.0
136,USC00519397,2010-06-04,0.00,76.0
137,USC00519397,2010-06-05,0.00,77.0


In [11]:
# Create database for December weather 
month = dt.datetime(2010,12,1).month

december_df = pd.DataFrame(columns = ["station_id", "date", "precipitation (inch)", "temperature (F)"])

for index in np.arange(0, len(weather_df), 1):
    if weather_df.iloc[index]['date'].month == month:
        row = weather_df.iloc[index]
        december_df = december_df.append(row)
    else: 
        december_df = december_df
        
december_df.head()
len(december_df["precipitation (inch)"])


1517

In [12]:
# Obtain statistics using the describe() function
june_df.describe()


,precipitation (inch),temperature (F)
count,1574.000000,1700.000000
mean,0.136360,74.944118
std,0.335731,3.257417
min,0.000000,64.000000
25%,0.000000,73.000000
50%,0.020000,75.000000
75%,0.120000,77.000000
max,4.430000,85.000000


In [13]:
# Obtain statistics using the describe() function
december_df.describe()

,precipitation (inch),temperature (F)
count,1405.000000,1517.000000
mean,0.216819,71.041529
std,0.541399,3.745920
min,0.000000,56.000000
25%,0.000000,69.000000
50%,0.030000,71.000000
75%,0.150000,74.000000
max,6.420000,83.000000


In [14]:
# Challenge 9 Summary: 
# A statistical analysis was performed on the Hawaii weather to determine whether or not it would be a good idea to invest in an
# ice cream and surf shop. Two specific months were analyzed over a period from 2010 to 2017, where rainfall and temperature data 
# were investigated. 
#
# As shown in the above cells, the June database contained 1574 and 1700 non-null values for
# precipitation and temperature respectively. The average rainfall for the June months was 0.136 inches with a standard 
# deviation of 0.336 inches. The average temperature for this period was 74.94 F, with a standard deviation of 3.26 F. 
# 
# The statistical description for the December database shows that there are 1405 and 1517 non-null values for 
# precipitation and temperature respectively. The corresponding average rainfall for 
# December month is 0.136 inches with a standard deviation of 0.541 inches. The average temperature was 71.0 F with a 
# standard deviation of 71.0 F. 
#
# The purpose of this analysis was to determine the weather patterns throughout all of the weather 
# stations in Hawaii. As shown in the results, the average rainfalls were quite similar, however when looking at the 
# maximum values, it can be shown that the December months hold more severe rain events than the June months. Additionally 
# when considering temperature, the June months are on average warmer than the December months. The summer months seem to be 
# more desirebale for business operation as the total rainfall and temperature are more desireable for surfers. 
#
# Further analysis is required to ensure that the most accurate and profitable decision is being made. The following areas are 
# reccomended for further analysis: 
# 1. The impact of warming climates should be included in this investigation. As increasing global temperatures occurs, the weather 
# patterns begin to change and as a result this venture may become less desireable/economical in the future. Temperature and rainfall 
# patterns versus time should be analyzed to help with this decision. 
# 2. Surfers desire a high tide in order to maximize wave height and intensity, so in order to maximize surfer turnout, the rainfall 
# and temperature versus time of day should be considered. 